In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('rec-sys-data-req/movies_metadata.csv')

C:\Users\UMMAR\AppData\Local\Temp\ipykernel_11004\656259420.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('rec-sys-data-req/movies_metadata.csv')


lets check how many movies and what are te different columns available in dataset by isung shape method.

In [2]:
data.shape

(45466, 24)

In [3]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
#Only keep those features that we require 
data = data[['title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]

data.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [5]:
data['release_date'].head()

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object

In [6]:
data['release_date'].dtype

dtype('O')

this is in string object datatype, lets convert it into datetime object, for that we can use to_datetime method from pandas

In [7]:
#Convert release_date into pandas datetime format
data['release_date'] = pd.to_datetime(data['release_date'], errors='coerce')



next thing is seperation of year from date, i want to use lambda function and str.split method for this

In [8]:
#Extract year from the datetime
data['year'] = data['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [9]:
#Helper function to convert NaT to 0 and all other years to integers.
def convert_int(x):
    try:
        return int(x)
    except:
        return 0

In [10]:
#Apply convert_int to the year feature
data['year'] = data['year'].apply(convert_int)

We do not require the release_date feature anymore. So let's go ahead and remove it:

In [11]:
#Drop the release_date column
data = data.drop('release_date', axis=1)

#Display the dataframe
data.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


second data wragling is runtime, so lets check the runtime datatype

In [12]:
data['runtime'].dtype

dtype('float64')

The runtime column is already in a float that is usable. It doesn't require any additional wrangling. Let us now do data wrangling to  genres.

## Genres

lets look at the first item of genres, to check what structure it contains

In [13]:
#Print genres of the first movie
data.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [14]:
#Import the literal_eval function from ast
from ast import literal_eval

suppose here a is string object but it is a list. so apply literal eval on string and check the type of a

In [15]:
#Define a stringified list and output its type
a = "[1,2,3]"
print(type(a))

#Apply literal_eval and output type
b = literal_eval(a)
print(type(b))

<class 'str'>
<class 'list'>


In [16]:
#Convert all NaN into stringified empty lists
data['genres'] = data['genres'].fillna('[]')

#Apply literal_eval to convert stringified empty lists to the list object
data['genres'] = data['genres'].apply(literal_eval)

In [17]:
data['genres'].dtype

dtype('O')

In [18]:
data.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


In [19]:
#Convert list of dictionaries to a list of strings
data['genres'] = data['genres'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])

In [20]:
data.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995


In [21]:
pd.Series(data.iloc[0]['genres'])

0    animation
1       comedy
2       family
dtype: object

In [23]:
#Create a new feature by exploding genres
s = data.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

#Name the new feature as 'genre'
s.name = 'genre'

#Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'.
req_df = data.drop('genres', axis=1).join(s)

#Print the head of the new gen_df
req_df.head()

C:\Users\UMMAR\AppData\Local\Temp\ipykernel_11004\1054843366.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = data.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,animation
0,Toy Story,81.0,7.7,5415.0,1995,comedy
0,Toy Story,81.0,7.7,5415.0,1995,family
1,Jumanji,104.0,6.9,2413.0,1995,adventure
1,Jumanji,104.0,6.9,2413.0,1995,fantasy


In [25]:
def knowledge_rec(req_df, percentile=0.8):
    #Ask for preferred genres
    print("Input preferred genre")
    genre = input()
    
    #Ask for lower limit of duration
    print("Input shortest duration")
    low_time = int(input())
    
    #Ask for upper limit of duration
    print("Input longest duration")
    high_time = int(input())
    
    #Ask for lower limit of timeline
    print("Input earliest year")
    low_year = int(input())
    
    #Ask for upper limit of timeline
    print("Input latest year")
    high_year = int(input())
    
    #Define a new movies variable to store the preferred movies. Copy the contents of gen_df to movies
    movies = req_df.copy()
    
    #Filter based on the condition
    movies = movies[(movies['genre'] == genre) & 
                    (movies['runtime'] >= low_time) & 
                    (movies['runtime'] <= high_time) & 
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]
    
    #Compute the values of C and m for the filtered movies
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    
    #Only consider movies that have higher than m votes. Save this in a new dataframe rec_movies
    rec_movies = movies.copy().loc[movies['vote_count'] >= m]
    
    #Calculate score using the IMDB formula
    rec_movies['score'] = rec_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                       + (m/(m+x['vote_count']) * C)
                                       ,axis=1)

    #Sort movies in descending order of their scores
    rec_movies = rec_movies.sort_values('score', ascending=False)
    
    return movies
    

Ultimately we created a knowledge rec fucntion and we will put our model into action.

In [26]:
#Generate the chart for top animation movies and display top 5.
knowledge_rec(req_df).head()

Input preferred genre
action
Input shortest duration
120
Input longest duration
150
Input earliest year
2005
Input latest year
2010


,title,runtime,vote_average,vote_count,year,genre
9888,Sin City,124.0,7.2,2755.0,2005,action
9892,Sahara,124.0,5.7,440.0,2005,action
10012,Kingdom of Heaven,144.0,6.6,1187.0,2005,action
10069,Star Wars: Episode III - Revenge of the Sith,140.0,7.1,4200.0,2005,action
10104,Mr. & Mrs. Smith,120.0,6.5,3048.0,2005,action
